### Importing the Needed Modules

In [0]:
import sys
import os

# Absolute path to the repo root
PROJECT_ROOT = "/Workspace/Users/thiruvengadamk16@gmail.com/Retail-And-Ecommerce-Analytics-Platform"

# Add repo root to PYTHONPATH (only once)
if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

# Debug checks (safe to remove later)
print("Current working directory:", os.getcwd())
print("Repo root added to path:", PROJECT_ROOT in sys.path)
from src.paths import *

In [0]:
import os
from pyspark.sql.functions import *
from pyspark.sql.types import *
from delta.tables import DeltaTable
import sys
sys.path.append("/Workspace/Users/thiruvengadamk16@gmail.com/Retail-And-Ecommerce-Analytics-Platform")

from src.paths import BRONZE_CUSTOMERS_PATH, SILVER_CUSTOMERS_PATH
from src.schema_definitions import SILVER_CUSTOMERS_SCHEMA

### Querying the Bronze Customers Table

In [0]:
cust_bronze_df = spark.read.table(BRONZE_CUSTOMERS_PATH)
cust_bronze_df.limit(5).display()

### Silver Customers Schema Reference

In [0]:
SILVER_CUSTOMERS_SCHEMA

### Schema Enforcement and Column Name Standardizing

In [0]:
cust_silver_df = cust_bronze_df.select(
    col("Customer ID").cast(IntegerType()).alias("customer_id"),
    col("Name").cast(StringType()).alias("name"),
    col("Email").cast(StringType()).alias("email"),
    col("Telephone").cast(StringType()).alias("telephone"),
    col("City").cast(StringType()).alias("city"),
    col("Country").cast(StringType()).alias("country"),
    col("Gender").cast(StringType()).alias("gender"),  
    col("Date Of Birth").cast(DateType()).alias("date_of_birth"),
    col("Job Title").cast(StringType()).alias("job_title"),
    col("ingestion_ts").cast(TimestampType()),
    col("_source_file").cast(StringType())
)


### Cleaning Empty spaces and Formatting Values

In [0]:
cust_silver_df = (
    cust_silver_df.withColumn("name", initcap(trim(col("name"))))
    .withColumn("email", lower(trim(col("email"))))
    .withColumn("city", initcap(trim(col("city"))))
    .withColumn("country", upper(trim(col("country"))))
    .withColumn("gender", when((lower(trim(col("gender"))) == "m") | (lower(trim(col("gender"))) == "male"), "Male")
        .when((lower(trim(col("gender"))) == "f") | (lower(trim(col("gender"))) == "female"), "Female")
        .otherwise("Unknown"))
    .withColumn("job_title", initcap(trim(col("job_title"))))
)

In [0]:
cust_silver_df.limit(5).display()

### Dropping Duplicate Rows

In [0]:
print(f"Before deduplicate count : {cust_silver_df.count()}")
cust_silver_df = cust_silver_df.dropDuplicates(['customer_id', 'name', 'email'])
print(f"After deduplicate count : {cust_silver_df.count()}")

### Filtering out null customer_id, name, and email rows

In [0]:
cust_silver_df = cust_silver_df.filter('(customer_id is not null) and (name is not null) and (email is not null)')

### validating the primary Key

In [0]:
cust_silver_df.groupBy("customer_id").agg(count('*').alias("count")).filter('count > 1').display()

### Null Check

In [0]:
null_counts = cust_silver_df.select([sum(col(c).isNull().cast(IntegerType())).alias(c) for c in cust_silver_df.columns])
display(null_counts)

### Handling Null Values

In [0]:
cust_silver_df = cust_silver_df.fillna("UnKnown", ["city", "country",  "gender", "job_title"])

### Validating Nulls

In [0]:
null_counts = cust_silver_df.select([sum(col(c).isNull().cast(IntegerType())).alias(c) for c in cust_silver_df.columns])
display(null_counts)

### Schema Enforcement Check

In [0]:
expected_cols = set(SILVER_CUSTOMERS_SCHEMA.keys())
incoming_cols = set(cust_silver_df.columns)

unknown_cols = incoming_cols - expected_cols - {
    "ingestion_ts", "_source_file"
}

print("Unknown columns in Bronze:", unknown_cols)


### Creating or Updating Silver Customers Table

In [0]:
if not spark.catalog.tableExists(SILVER_CUSTOMERS_PATH):
    (
        cust_silver_df.write
        .format("delta")
        .mode("overwrite")
        .saveAsTable(SILVER_CUSTOMERS_PATH)
    )
else:
    cust_silver_tbl = DeltaTable.forName(spark, SILVER_CUSTOMERS_PATH)

    cust_silver_tbl.alias("tgt").merge(
        cust_silver_df.alias("src"),
        "tgt.customer_id = src.customer_id"
    ).whenMatchedUpdateAll() \
    .whenNotMatchedInsertAll() \
    .execute()

In [0]:
spark.read.table(SILVER_CUSTOMERS_PATH).limit(5).display()

In [0]:
spark.read.table(SILVER_CUSTOMERS_PATH).count()